In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from tqdm import tqdm

# 시드 설정
SEED = 456
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# 디바이스 설정
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 모델과 토크나이저 로드 함수
def load_model_and_tokenizer(model_name, num_labels=7):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(DEVICE)
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    return model, tokenizer

# 데이터 로드
noise_data = pd.read_csv("noise_train.csv")
clean_data = pd.read_csv("clean_train.csv")

# 최대 입력 길이 설정
max_length = 128  # 모델에 따라 맞는 값으로 설정

# 모델과 토크나이저 로드 (네 개의 서로 다른 모델 사용)
models_and_tokenizers = [
    load_model_and_tokenizer("klue/bert-base"),
    load_model_and_tokenizer("klue/roberta-base"),
    load_model_and_tokenizer("klue/roberta-large"),
    load_model_and_tokenizer("klue/roberta-small")
]

# 텍스트와 레이블을 학습에 사용될 형태로 변환
def tokenize_function(examples, tokenizer):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_length)

# 학습용 데이터셋 준비
train_dataset = Dataset.from_pandas(noise_data[['text', 'target']])
train_dataset = train_dataset.map(lambda x: tokenize_function(x, models_and_tokenizers[0][1]), batched=True)
train_dataset = train_dataset.rename_column("target", "labels")

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=9,
    weight_decay=0.01,
    seed=SEED,
)

# 각 모델 학습
trainers = []
for model, tokenizer in models_and_tokenizers:
    tokenized_train_dataset = train_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
    )
    trainer.train()
    trainers.append(trainer)

# Clean Data 재라벨링 준비
clean_dataset = Dataset.from_dict({"text": clean_data['text'].tolist()})
clean_dataset = clean_dataset.map(lambda x: tokenize_function(x, models_and_tokenizers[0][1]), batched=True)

# 예측 함수 (모델 앙상블)
def relabel_texts_ensemble(dataset):
    predictions = []
    for trainer in trainers:
        pred = trainer.predict(dataset).predictions
        predictions.append(pred)
    # 모든 모델의 예측을 평균하여 최종 예측 생성
    ensemble_predictions = sum(predictions) / len(predictions)
    predicted_labels = ensemble_predictions.argmax(axis=1)
    return predicted_labels

# 재라벨링 수행
clean_data['target'] = relabel_texts_ensemble(clean_dataset)

# 데이터 저장
final_data = clean_data[['ID', 'text', 'target']]
final_data.to_csv("relabel_clean_train_m4.csv", index=False)

In [3]:
import pandas as pd

# 파일 로드
noise_data = pd.read_csv("noise_train.csv")
relabel_clean_data = pd.read_csv("relabel_clean_train_m4.csv")

combined_data = pd.concat([noise_data, relabel_clean_data], ignore_index=True)

# 합쳐진 데이터 확인 및 저장
print(combined_data.head())
combined_data.to_csv("noise_and_relabeled_clean_train.csv", index=False)

                    ID                              text  target
0  ynat-v1_train_00000  정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보       4
1  ynat-v1_train_00001       K찰.국DLwo 로L3한N% 회장 2 T0&}송=       3
2  ynat-v1_train_00002            m 김정) 자주통일 새,?r열1나가야1보       2
3  ynat-v1_train_00004      pI美대선I앞두고 R2fr단 발] $비해 감시 강화       6
4  ynat-v1_train_00006             프로야구~롯TKIAs광주 경기 y천취소       1
